In [1]:
#reddit api

import praw

In [20]:
#download method
import requests

#other
import pprint
import re
from datetime import date
import time

# File management
import os, os.path
import json

In [17]:
def downloadImage(imageUrl, localFileName):
    response = requests.get(imageUrl)
#     print(imageUrl)
#     if response.status_code == 200:
#         print('Downloading %s...' % (localFileName))
    with open(localFileName, 'wb') as fo:
        for chunk in response.iter_content(4096):
            fo.write(chunk)

In [4]:
#https://stackoverflow.com/questions/8032642/
# how-to-obtain-image-size-using-standard-python-class-without-using-external-lib/20380514#20380514
import struct
import imghdr

def get_image_size(fname):
    '''Determine the image type of fhandle and return its size.
    from draco'''
    with open(fname, 'rb') as fhandle:
        head = fhandle.read(24)
        if len(head) != 24:
            return
        if imghdr.what(fname) == 'png':
            check = struct.unpack('>i', head[4:8])[0]
            if check != 0x0d0a1a0a:
                return
            width, height = struct.unpack('>ii', head[16:24])
        elif imghdr.what(fname) == 'gif':
            width, height = struct.unpack('<HH', head[6:10])
        elif imghdr.what(fname) == 'jpeg':
            try:
                fhandle.seek(0) # Read 0xff next
                size = 2
                ftype = 0
                while not 0xc0 <= ftype <= 0xcf:
                    fhandle.seek(size, 1)
                    byte = fhandle.read(1)
                    while ord(byte) == 0xff:
                        byte = fhandle.read(1)
                    ftype = ord(byte)
                    size = struct.unpack('>H', fhandle.read(2))[0] - 2
                # We are at a SOFn block
                fhandle.seek(1, 1)  # Skip `precision' byte.
                height, width = struct.unpack('>HH', fhandle.read(4))
            except Exception: #IGNORE:W0703
                return
        else:
            return
        return width, height

In [5]:
reddit = praw.Reddit(client_id='NLIznGitXU82bA',
                     client_secret='in6SUevIRf3dLYHqu2YFH8xYAq8',
                     password='willmarkandmatt',
                     user_agent='script2',
                     username='231ntest1')

In [6]:
print(reddit.user.me())

231ntest1


In [7]:
print(reddit.read_only)  # Output: False

False


In [8]:
for submission in reddit.subreddit('pics').hot(limit=10):
    print(submission.title)

May 2017 Transparency Report
Helping your parents with a tech problem
Giza, Egypt.
I want someone to look at me the way peter Dinklage looks at his wax self
Buddy visited Trejo's new donut shop in LA yesterday (national donut day)
Lake Bled
we can not say we were not warned .
Steam Powered Motorcycle. I particularly like the 'bell' behind the headlight!
Pay it forward.
Perfection in simplicity


In [54]:
subName = "aww"
substats ={
    "title" : "rarepup",
    "comments" : ["wow", "such rare", "so pup"]
}
json.dumps(substats)
a = "hello? yes. it's me!"
re.split("[?.!]", a)



['hello', ' yes', " it's me", '']

In [24]:
# Extract comment + image pairs from subreddit

info = {
    "year" : 2017,
    "version" : 0,
    "description" : "CS231N Toy Dataset",
    "contributor" : "Mathieu Rolfo, Mark Nishimura, William Clary",
    "url" : "",
    "date_created" : "",
}

unlicensed_license = {
    "id" : 0,
    "name" : "Unlicensed",
    "url" : "",
}

licenses = [unlicensed_license]
images = []
annotations = []
nposts = 10
commentsperpost = 5
subreddits = ["rarepuppers", "aww"]
captionID = 0
now = time.time()
for subName in subreddits:
    print("Extracting data from r/" + subName + "...")
    subImages = []
    subCaptions = []
    for submission in reddit.subreddit(subName).hot(limit=nposts):
        #pprint.pprint(vars(submission))
    #     print(submission.comments)
        submission.comments.replace_more(limit=0)
    #     for top_level_comment in submission.comments[:5]:
    #         # Extract first sentence of each top comment.
    #         print(re.split('[.!?]',top_level_comment.body)[0]) 
    
        # Filter out gifs, albums
        if not submission.url.endswith(('jpg')):
            continue
        # Download image and attempt to extract width and height:
        filename = submission.id + ".jpg"
        downloadImage(submission.url, os.path.join("raw", filename))
        size = get_image_size(os.path.join("raw", filename))
        if size is None:
            continue
            
        # Save this image
        width, height = size
        os.rename(os.path.join("raw", filename), os.path.join(subName, filename)) # Move to new location

        # Extract imageID from base 36
        imageID = int(submission.id, 36)

        # Generate Image and Annotation json
        # Get top n comments:
        captions = [
            {
                "id" : captionID + k,
                "image_id" : imageID,
                "caption" : re.split('[.!?\n]',top_level_comment.body)[0],
            }
            for k, top_level_comment in enumerate(submission.comments[:commentsperpost])
        ]
        captionID += commentsperpost

        subCaptions += captions
        image = {
            "id" : imageID,
            "width" : width,
            "height" : height,
            "file_name" : filename,
            "license" : 0,
            "flickr_url" : "",
            "coco_url" : "",
            "date_captured" : str(date.today()),
        }
        subImages += [image]
    print("\tRetrieved "+ str(len(subImages)) + " images from r/" + subName + ".")
    print("\tRetrieved "+ str(len(subCaptions)) + " captions from r/" + subName + ".")
    images+= subImages
    annotations += subCaptions
    
out = {
    "info" : info,
    "images" : images,
    "annotations" : annotations,
    "licenses" : licenses
}
print("Retrieved "+ str(len(images)) + " total images.")
print("Retrieved "+ str(len(annotations)) + " total captions")
print("Done. (" + str(time.time() - now) + " s)")

Extracting data from r/rarepuppers...
	Retrieved 4 images from r/rarepuppers.
	Retrieved 16 captions from r/rarepuppers.
Extracting data from r/aww...
	Retrieved 7 images from r/aww.
	Retrieved 35 captions from r/aww.
Retrieved 11 total images.
Retrieved 51 total captions
Done. (32.4647519588 s)


In [10]:
with open('captions_train.json', 'w') as outfile:
    json.dump(out, outfile)

In [17]:
client_id = 'b66557d3975b3dc'
client_secret = '32ca75602eb1f201ff77ecd96c25134da143c010'
client = ImgurClient(client_id, client_secret)

In [22]:
out

{'annotations': [{'caption': u"I'd like 2 G O O D B O Y E S pls ",
   'id': 0,
   'image_id': 388133459},
  {'caption': u'L O S S L E A D E R B O Y E S ',
   'id': 1,
   'image_id': 388133459},
  {'caption': u'no fren whatr u doin', 'id': 5, 'image_id': 388171075},
  {'caption': u'AAAAAAAAAAAAAAAAAAAAAAAAAAAAHHHHHHHHH\n\n-pupper probably',
   'id': 6,
   'image_id': 388171075},
  {'caption': u'what is ball', 'id': 7, 'image_id': 388171075},
  {'caption': u'halp plz', 'id': 8, 'image_id': 388171075},
  {'caption': u'She sure looks heckin refreshed',
   'id': 10,
   'image_id': 388032667},
  {'caption': u'Splish splash doggo', 'id': 11, 'image_id': 388032667},
  {'caption': u'Serenegirle enjoys day at beach :) ',
   'id': 12,
   'image_id': 388032667},
  {'caption': u'What a heckin adorable pupperina',
   'id': 13,
   'image_id': 388032667},
  {'caption': u'###[that pupper is a mermaid now](#bork)',
   'id': 14,
   'image_id': 388032667},
  {'caption': u'WoaH he is doin me a scare', 'id'

In [9]:
int("6ew541", 36)

387811297

In [23]:
re.split("[\n]", "hi\nthere")

['hi', 'there']

In [44]:
str(date.today())

'2017-06-03'

In [ ]:
ann = {
"info" : info,
"images" : [image],
"annotations" : [annotation],
"licenses" : [license],
}





